In [ ]:
json_path = "./data/raw-train/tiger-coco.json"
img_dir = "./data/raw-train/"
img_dir_precise = "./data/raw-train/images"
crop_rois_dir = "./data/processed-train/cropped/"
processed_dir = "./data/processed-train/"

In [ ]:
from src.image_preprocessor import ImageProcessor, InplaceOption

image_processor = ImageProcessor("./data/raw-test/images")

In [ ]:
target_images = image_processor.select_random_images()
image_processor.normalize(target_images, out_dir=processed_dir, inplace=True, inplace_option=InplaceOption.HEMATOXYLIN)
image_processor.equalize_hist(inplace=True, out_dir="./data/processed-train/equalized_hist")

In [ ]:
image_processor.relabel_binary(masks_dir="./data/raw-test/masks", out_dir="./data/processed-test/masks_binary", target_class=2)

In [ ]:
image_processor.scale(masks_dirs=["./data/processed-test/masks_binary"], out_dir="./data/processed-test/scaled", inplace=True)

In [ ]:
image_processor.extract_patches(masks_dirs=["./data/processed-test/scaled/masks/"], out_dir="./data/processed-test/patches")

In [ ]:
from src.image_stats import ImageStats

img_stats = ImageStats("./data/processed-train/patches/images")
img_stats.get_image_stats(visual=True)

In [ ]:
std = img_stats.contrast_std()
michelson = img_stats.michelson_contrast()

In [1]:
from src.mask_generator import MaskGenerator, MaskOperations

mask_generator = MaskGenerator("./data/raw-train/images", json_file_path="./data/raw-train/tiger-coco.json")

In [2]:
mask_generator._fuse_masks(masks_dir="./data/processed-train/masks", out_dir="./data/processed-train/masks_fused", colorized_dir="./data/processed-train/masks_fused_color")

In [2]:
from src.sample import Sample

sample = Sample(img_path="./data/raw-train/images/TCGA-EW-A1P8-01Z-00-DX1.E9852193-8CDD-49EF-B49B-DA6931198F0D_[8391, 13690, 8532, 13838].png", mask_path="./data/processed-train/masks/fused_exclude_1/TCGA-EW-A1P8-01Z-00-DX1.E9852193-8CDD-49EF-B49B-DA6931198F0D_[8391, 13690, 8532, 13838].png")

In [3]:
sample.save_overlay("./data/processed-train/for_presentation")

In [4]:
sample.save_with_bbox(json_path="./data/raw-train/tiger-coco.json",out_dir="./data/processed-train/for_presentation")

In [9]:
sample_test = Sample(img_path="./data/raw-test/images/10_1.png", mask_path="./data/processed-test/binary_masks/10_1.png")

In [10]:
sample_test.save_overlay("./data/processed-train/for_presentation")

In [19]:
import segmentation_models_pytorch as smp
from torchsummary import summary

model = smp.Unet(
        encoder_name = "resnet34",
        encoder_weights = "imagenet",
        in_channels = 3,
        classes = 1,
        learning_rate = 0.001         
)

In [20]:
print(model)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [21]:
sum = summary(model, (3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 3, 128, 128]         --
|    └─Conv2d: 2-1                       [-1, 64, 64, 64]          9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 64, 64]          128
|    └─ReLU: 2-3                         [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-4                    [-1, 64, 32, 32]          --
|    └─Sequential: 2-5                   [-1, 64, 32, 32]          --
|    |    └─BasicBlock: 3-1              [-1, 64, 32, 32]          73,984
|    |    └─BasicBlock: 3-2              [-1, 64, 32, 32]          73,984
|    |    └─BasicBlock: 3-3              [-1, 64, 32, 32]          73,984
|    └─Sequential: 2-6                   [-1, 128, 16, 16]         --
|    |    └─BasicBlock: 3-4              [-1, 128, 16, 16]         230,144
|    |    └─BasicBlock: 3-5              [-1, 128, 16, 16]         295,424
|    |    └─BasicBlock: 3-6              [-1, 128, 16, 16] 